In [13]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
glass_identification = fetch_ucirepo(id=42)

# data (as pandas dataframes)
X = glass_identification.data.features
y = glass_identification.data.targets
X


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0
...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0


In [14]:
import numpy as np

y_array = np.array(y)

if np.isnan(y_array).any():
    print("В переменной y есть NaN значения.")
else:
    unique, counts = np.unique(y_array, return_counts=True)
    class_distribution = dict(zip(unique, counts))
    print("Распределение классов:", class_distribution)


Распределение классов: {1: 70, 2: 76, 3: 17, 5: 13, 6: 9, 7: 29}


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Масштабирование данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение масштабированных данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Обучение модели на исходных данных с увеличенным количеством итераций
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train.values.ravel()) # Используем values.ravel() здесь

# Предсказание и оценка модели
y_pred = model.predict(X_test)
print("Классификационный отчет для исходных данных:")
print(classification_report(y_test, y_pred, zero_division=1))

Классификационный отчет для исходных данных:
              precision    recall  f1-score   support

           1       0.64      0.74      0.68        19
           2       0.62      0.70      0.65        23
           3       0.00      0.00      0.00         4
           5       1.00      0.33      0.50         6
           6       0.67      0.67      0.67         3
           7       0.91      1.00      0.95        10

    accuracy                           0.68        65
   macro avg       0.64      0.57      0.58        65
weighted avg       0.67      0.68      0.65        65



In [23]:
"""
Мы видим, что общая точность упала немного после балансировки классов, но средний F1-балл увеличился, что указывает на улучшение сбалансированности производительности модели по разным классам. Важно отметить, что взвешенный средний F1-балл остался примерно на том же уровне.
Если важно равномерно распознавать все классы, модель после балансировки может быть предпочтительнее. Если же важнее общая точность и ваши основные классы уже хорошо представлены в данных, возможно, стоит остановиться на модели до балансировки и искать способы улучшить распознавание менее представленных классов без ухудшения результатов по основным классам.
"""

# Применение SMOTE к обучающим данным
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train.values.ravel()) # Используем values.ravel() здесь

# Обучение модели на сбалансированных данных
model.fit(X_train_resampled, y_train_resampled)

# Предсказание и оценка модели
y_pred_resampled = model.predict(X_test)
print("Классификационный отчет после балансировки классов:")
print(classification_report(y_test, y_pred_resampled, zero_division=1))

Классификационный отчет после балансировки классов:
              precision    recall  f1-score   support

           1       0.62      0.68      0.65        19
           2       0.69      0.39      0.50        23
           3       0.20      0.50      0.29         4
           5       0.80      0.67      0.73         6
           6       0.75      1.00      0.86         3
           7       0.83      1.00      0.91        10

    accuracy                           0.63        65
   macro avg       0.65      0.71      0.65        65
weighted avg       0.67      0.63      0.63        65

